In [1]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langchain_unstructured import UnstructuredLoader
from langchain_core.documents import Document
from typing import List 
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_experimental.text_splitter import SemanticChunker
import weaviate 
from weaviate.classes.config import Configure, Property, DataType
from weaviate.util import generate_uuid5
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()


True

# 1. Load document (texe, PDF, Markdown, Web,..)

In [2]:
def load_documents_format(file_path: str) -> List[Document]:
    """
    Load documents using the best loader for each file format.
        
    Returns:
        List of Document objects with page_content and metadata
    """
    path = Path(file_path)
    file_ext = path.suffix.lower()
    
    # PDF files - one Document per page
    if file_ext == '.pdf':
        loader = PyPDFLoader(file_path)
        docs = loader.load()
        return docs
    
    # Word documents - preserves formatting
    elif file_ext == '.docx' or file_ext == '.md':
        loader = UnstructuredLoader(file_path)
        docs = loader.load()
        return docs
    
    # Text/markdown files
    elif file_ext == '.txt':
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return [Document(
            page_content=content,
            metadata={
                "source": str(path),
                "file_type": "text"
            }
        )]
    
def load_multiple_documents(
    directory_path: str,
    file_types: List[str] = None,
    verbose: bool = False
) -> List[Document]:
    """
    Load all documents from a directory using optimal loaders.
        
    Returns:
        List of all loaded Documents
    """
    if file_types is None:
        file_types = ['.pdf', '.docx', '.txt', '.md']
    
    directory = Path(directory_path)
    all_documents = []
    
    for file_type in file_types:
        for file_path in directory.glob(f'*{file_type}'):
            try:
                if verbose:
                    print(f"Loading {file_path.name}...")
                docs = load_documents_format(str(file_path))
                
                all_documents.extend(docs)
                if verbose:
                    print(f"✓ Loaded {len(docs)} documents")
            except Exception as e:
                print(f"✗ Error loading {file_path.name}: {e}")
    
    return all_documents

In [3]:
documents = load_multiple_documents('documents/', verbose=True)

Loading document.pdf...
✓ Loaded 6 documents


In [5]:
documents[0].metadata

{'producer': 'Skia/PDF m144 Google Docs Renderer',
 'creator': 'PyPDF',
 'creationdate': '',
 'title': 'Nghiệp vụ quản lí project',
 'source': 'documents\\document.pdf',
 'total_pages': 6,
 'page': 0,
 'page_label': '1'}

In [4]:
print(f'Number of documents: {len(documents)}')

Number of documents: 6


In [6]:
for doc in documents[:3]:
    print(f'Content: {doc.page_content}')  
    print(f'Metadata: {doc.metadata}')
    print('-' * 80)

Content: TÀI  LIỆU  QUY  CHUẨN  VẬN  HÀNH  &  PHỐI  
HỢP
 
LIÊN
 
BỘ
 
PHẬN
 
(SOP)
 
(V/v:  Quản  lý  Dự  án  và  Quy  trình  Xử  lý  Công  việc)  
THÔNG  TIN  TÀI  LIỆU  
Hạng  mục  Chi  tiết  
Mã  tài  liệu  SOP-PM-2024-01  
Phiên  bản  1.0  
Ngày  ban  hành  24/05/2024  
Phạm  vi  áp  dụng  Toàn  bộ  các  phòng  ban:  BOD,  Sales,  PMO,  Product,  Tech,  QA/QC  
Người  phê  duyệt  [Tên  Giám  Đốc/CEO]  
MỤC  LỤC  
1.  Tổng  quan  và  Mục  đích 2.  Định  nghĩa  thuật  ngữ 3.  Cơ  cấu  tổ  chức  &  Ma  trận  trách  nhiệm  (RACI) 4.  Quy  trình  Vòng  đời  Dự  án  (Project  Lifecycle) 5.  Quy  trình  Quản  lý  Task  (Task  Workflow) 6.  Cơ  chế  Phối  hợp  &  Giao  tiếp  Liên  bộ  phận 7.  Quản  lý  Thay  đổi  (Change  Requests) 8.  Hệ  thống  Công  cụ  &  KPIs 
1.  TỔNG  QUAN  VÀ  MỤC  ĐÍCH  1.1.  Mục  đích  
Tài  liệu  này  được  thiết  lập  nhằm:  ●  Chuẩn  hóa  quy  trình  làm  việc  từ  khâu  tiếp  nhận  yêu  cầu  đến  khi  bàn  giao  sản  phẩm.  ●  Xác  định  rõ  vai  trò,  trác

# 3. Chunking
    - Fixed 
    - Recursive ("\n\n", "\n", " ", "")
    - Semantic
    - Markdown character
    - LLM-based (context-aware)

In [4]:
def chunk_documents(
    documents: List[Document],
    strategy: str = 'recursive',
    chunk_size: int = 500,
    chunk_overlap: int = 50
) -> List[Document]:
        
        chunks = []
        if strategy == 'fixed':
            text_splitter = CharacterTextSplitter(
                chunk_size=chunk_size, 
                chunk_overlap=chunk_overlap
            )
            chunks = text_splitter.split_documents(documents)
            
        elif strategy == "recursive":
            splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap,
                separators=["\n\n", "\n", " ", ""]
            )
            chunks = splitter.split_documents(documents)
            
        elif strategy == 'markdown':
            header_splitter = MarkdownHeaderTextSplitter(
                headers_to_split_on=[
                    ("#", "Header 1"),
                    ("##", "Header 2"),
                    ("###", "Header 3"),
                ]
            )
            chunks = header_splitter.split_documents(documents)
        
        elif strategy == 'semantic':
            embeddings = OpenAIEmbeddings()
            text_splitter = SemanticChunker(
                embeddings=embeddings,
                breakpoint_threshold_type="percentile"
            )
            chunks = text_splitter.split_documents(documents)
        
        elif strategy == 'context-aware':
            splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size, 
                chunk_overlap=chunk_overlap, 
                separators=["\n\n", "\n", " ", ""]
            )
            chunks = splitter.split_documents(documents)
            
            # Add context description using LLM
            prompt_template = PromptTemplate.from_template(
"""You are an expert at summarizing information for retrieval.
                
Analyze the following text chunk and provide a brief, high-level context description.
Focus on the main topic, entities, or event described.

Text Chunk:
"{chunk_text}"

Context Description:"""
            )
            
            llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1.0)
            chain = prompt_template | llm | StrOutputParser()
            
            for chunk in tqdm(chunks):
                context = chain.invoke({'chunk_text': chunk.page_content})
                chunk.metadata['context_description'] = context.strip()
        
        else:
            raise ValueError(f"Unknown chunking strategy: {strategy}")
        
        return chunks

In [6]:
chunked_docs = chunk_documents(documents, strategy='recursive', chunk_size=500, chunk_overlap=50)

In [7]:
print(f"Number of chunks created: {len(chunked_docs)}")

Number of chunks created: 27


In [8]:
for i, doc in enumerate(chunked_docs[:3]):
    print(f"Chunk {i}: {doc.page_content}")
    print("-" * 80)

Chunk 0: TÀI  LIỆU  QUY  CHUẨN  VẬN  HÀNH  &  PHỐI  
HỢP
 
LIÊN
 
BỘ
 
PHẬN
 
(SOP)
 
(V/v:  Quản  lý  Dự  án  và  Quy  trình  Xử  lý  Công  việc)  
THÔNG  TIN  TÀI  LIỆU  
Hạng  mục  Chi  tiết  
Mã  tài  liệu  SOP-PM-2024-01  
Phiên  bản  1.0  
Ngày  ban  hành  24/05/2024  
Phạm  vi  áp  dụng  Toàn  bộ  các  phòng  ban:  BOD,  Sales,  PMO,  Product,  Tech,  QA/QC  
Người  phê  duyệt  [Tên  Giám  Đốc/CEO]  
MỤC  LỤC
--------------------------------------------------------------------------------
Chunk 1: MỤC  LỤC  
1.  Tổng  quan  và  Mục  đích 2.  Định  nghĩa  thuật  ngữ 3.  Cơ  cấu  tổ  chức  &  Ma  trận  trách  nhiệm  (RACI) 4.  Quy  trình  Vòng  đời  Dự  án  (Project  Lifecycle) 5.  Quy  trình  Quản  lý  Task  (Task  Workflow) 6.  Cơ  chế  Phối  hợp  &  Giao  tiếp  Liên  bộ  phận 7.  Quản  lý  Thay  đổi  (Change  Requests) 8.  Hệ  thống  Công  cụ  &  KPIs 
1.  TỔNG  QUAN  VÀ  MỤC  ĐÍCH  1.1.  Mục  đích
--------------------------------------------------------------------------------

# 4. Store to Vector DB
    - Connect to DB
    - Get/Create collection (vectorizer, reranker, properties(for metadata filtering))
    - Load doc to collection

In [9]:
# Connect to Weaviate instance
client = weaviate.connect_to_local(
    port=8080, 
    grpc_port=50051
)

In [17]:
collection_name = "ProjectDocuments"

# Check if collection exists
if client.collections.exists(collection_name):
    client.collections.delete(collection_name)
else:
    # Create new collection
    client.collections.create(
        name=collection_name,
        vectorizer_config=Configure.Vectorizer.text2vec_transformers(),
        reranker_config=Configure.Reranker.transformers(),
        properties=[
            Property(name='content', data_type=DataType.TEXT, vectorize_property=True),
            Property(name='source', data_type=DataType.TEXT, vectorize_property=False),
            Property(name='title', data_type=DataType.TEXT, vectorize_property=False),
            Property(name='creationdate', data_type=DataType.TEXT, vectorize_property=False),
            Property(name='author', data_type=DataType.TEXT, vectorize_property=False),
            Property(name='page', data_type=DataType.INT, vectorize_property=False)
        ]
    )

In [22]:
collection = client.collections.get(collection_name)
# Inser documents into the collection
with collection.batch.dynamic() as batch:
    for doc in tqdm(chunked_docs):
        properties = {
            'content': doc.page_content,
            'source': doc.metadata.get('source', ''),
            'title': doc.metadata.get('title', ''),
            'creationdate': doc.metadata.get('creationdate', ''),
            'author': doc.metadata.get('author', ''),
            'page': doc.metadata.get('page', '')
        }
        
        batch.add_object(
            properties=properties,
            uuid=generate_uuid5(doc.page_content)
        )

100%|██████████| 27/27 [00:00<00:00, 7601.44it/s]


In [23]:
# Check for errors
if len(collection.batch.failed_objects) > 0:
    print(f"Errors occurred: {len(collection.batch.failed_objects)} objects failed")
else:
    print("All documents stored successfully!")

All documents stored successfully!


In [24]:
results = collection.query.fetch_objects(limit=3, include_vector=True)
for obj in results.objects:
    print("Properties:", obj.properties)
    print("Vectors:", obj.vector)

Properties: {'title': 'Nghiệp vụ quản lí project', 'page': 5, 'creationdate': '', 'author': '', 'content': 'dẫn\n \nnhân\n \nviên\n \nthực\n \nhiện\n \nđúng\n \nquy\n \ntrình.\n \nSOẠN  THẢO  BỞI  PHÊ  DUYỆT  BỞI  (Ký  tên)  (Ký  tên)  Trưởng  phòng  PMO  Giám  Đốc  Điều  Hành', 'source': 'documents\\document.pdf'}
Vectors: {'default': [-0.22401610016822815, 0.11653438210487366, -0.14501255750656128, 0.04166174307465553, 0.10868892818689346, 0.06744257360696793, -0.17081943154335022, 0.09743505716323853, -0.11000503599643707, 0.14042377471923828, -0.13012908399105072, -0.001865580677986145, -0.02701362781226635, -0.20971937477588654, -0.18691906332969666, -0.04765103757381439, 0.00817616656422615, 0.10458318144083023, 0.09941302239894867, -0.03517815098166466, 0.1888446807861328, -0.06872246414422989, 0.05309649184346199, -0.1680295467376709, 0.017354793846607208, 0.17394335567951202, -0.145317941904068, -0.1509951651096344, 0.04413604736328125, -0.2563949525356293, 0.08842508494853973

In [25]:
result = collection.aggregate.over_all(total_count=True)
print(f"Total objects in collection: {result.total_count}")

Total objects in collection: 27


In [26]:
client.close()